In [2]:
#import libraries and data
import psycopg2
import pandas as pd

In [3]:
#connect to database oppurtunity_youth and create cursor
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME)
cursor = conn.cursor()

In [4]:
#create a list of the table names
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
tables = []
for table in cursor.fetchall():
    tables.append(table[0])
tables

['pums_2017',
 'puma_names_2010',
 'wa_jobs_2017',
 'wa_geo_xwalk',
 'ct_puma_xwalk']

In [18]:
# Filter puma_names_2010 to King County in Washington
# Filter pums_2017 for ages between 16 and 25
# Join pums_2017 with filtered puma_names_2010
# Create a data frame from a SQL query with info for puma, person, age, education, and work
# Column info:
    # PUMA          Public use microdata area code
    # puma_name     County, city, location
    # SERIALNO      Housing unit/GQ person serial number
    # sporder       Which person in housing unit
    # agep          Age
    # sch           school enrollment (1 = has not attended in last 3 months)
    # schl          Education level
    # esr           Employment status

df_puma = pd.read_sql(""" SELECT puma, puma_name, serialno, sporder, agep, sch, schl, esr
                    FROM puma_names_2010 pn
                    JOIN pums_2017 pms
                    USING (puma)                               /* Total 510,738 rows */  
                    WHERE state_name LIKE 'Washington%'        /* Washington State: 359,075 */
                    AND puma_name LIKE 'King%'                 /* King County: 66,673 */
                    AND agep BETWEEN 15.9 AND 25.0             /* YOUTH 6,911 */ 
                    AND (esr = '3' or esr = '6' OR  sch = '1') /* Employment Status Recode 3: unemployed 467 */
                    ;""", conn)

df_puma

,puma,puma_name,serialno,sporder,agep,sch,schl,esr
0,11616,"King County (Northeast)--Snoqualmie City, Cott...",2013000001693,02,25.0,1,22,6
1,11606,"King County (Northwest)--Shoreline, Kenmore & ...",2013000003218,02,21.0,1,16,1
2,11608,King County (Northwest Central)--Greater Belle...,2013000003370,03,18.0,2,18,6
3,11610,"King County (Central)--Renton City, Fairwood, ...",2013000003570,01,24.0,1,21,1
4,11610,"King County (Central)--Renton City, Fairwood, ...",2013000003570,02,24.0,1,21,1
...,...,...,...,...,...,...,...,...
5489,11616,"King County (Northeast)--Snoqualmie City, Cott...",2017001505038,01,24.0,1,16,1
5490,11613,King County (Southwest Central)--Kent City ...,2017001518359,01,18.0,1,11,6
5491,11606,"King County (Northwest)--Shoreline, Kenmore & ...",2017001526134,01,18.0,2,18,3
5492,11613,King County (Southwest Central)--Kent City ...,2017001530818,01,23.0,1,21,1
